In [452]:
import pandas as pd
import random
import googlemaps
from datetime import datetime

import tsp #solves the travelling salesman problem

class GoogleMapsClient():
    def __init__(self):
        
        self.gmaps = googlemaps.Client(key=os.environ['mapsAPI'])
        self.poi_latlon = None
    
    def get_place_ids_from_POIs(self,pois):
        #accept list of pois
        
        candidates = []
        
        for i in range(len(pois)):
            location_bias_string = f"point:{pois[i]['geoCode']['latitude']},{pois[i]['geoCode']['longitude']}"
            candidate = self.gmaps.find_place(input = pois[i]['name'], input_type='textquery',fields=['place_id'],location_bias=location_bias_string)
            
            candidates.append(candidate['candidates'][0]['place_id']) #extracts the first poi

        return candidates
    
    def get_mean_location(self,pois):
        #uses pois returned by amadeus
        poi_latlon = [pois[i]['geoCode'] for i in range(len(pois))]
        self.poi_latlon = pd.DataFrame.from_dict(poi_latlon).mean().to_dict()
        
        return  self.poi_latlon
        

    def generate_itinerary(self, pois, hotel_lat_lon = None):
        #assume hotel is dictionary of  in the style of {'latitude': 50.085133, 'longitude': 14.424776}
        #if not provided, the hotel is assumed to be at the mean location of the pois.
        #pois are list of pois generated by amadeus
        #result is a dictionary, that has:
        #    result['path'] is the path to be taken, indexed by pois
        #    result['travel_time'] is the matrix of travel times from a poi (row) to another poi (col). Note, indexes are shifted up by 1 due to hotel being added as index 0.
        #    result['dist_matrix'] is the distances matrix returned by google.
        
        if hotel_lat_lon is None:
            hotel_lat_lon = self.get_mean_location(pois)
            
        #convert pois to lat long
        poi_latlon = self.poi_latlon = [pois[i]['geoCode'] for i in range(len(pois))]
        #saved for future reference
        
        
        #collate
        full_latlon = [hotel_lat_lon] + poi_latlon
        num_of_points = len(full_latlon)
        
        #generate distance matrix from google
        dist_matrix = self.gmaps.distance_matrix(full_latlon,full_latlon,mode='transit')
        
        #generate summary of travel times (each number is the number of seconds of travel)
        travel_time = [[dist_matrix['rows'][o]['elements'][d]['duration']['value'] for d in range(num_of_points)] for o in range(num_of_points)]
        
        
        #convert to dictionary
        r = range(len(travel_time))
        cost = {(i,j): travel_time[i][j] for i in r for j in r}
        
        #solve travelling salesman problem
        (totaltime, path) = tsp.tsp(r, cost)
        
        #correct index, because hotel was added at point 0
        poi_path = [loc-1 for loc in path[1:]]    
        
        result = dict()
        result['path'] = poi_path
        result['dist_matrix'] = dist_matrix
        result['travel_time'] = travel_time
        
        return result
                                              
    def get_details(self, place_name,location_bias=None):
        #assumes the place_name is a text to query
        if location_bias is not None:
            candidate = self.gmaps.find_place(input = place_name, input_type='textquery',fields=['place_id'], location_bias_string=location_bias)
        else:
            candidate = self.gmaps.find_place(input = place_name, input_type='textquery',fields=['place_id'])
        
        try:
            place_id = candidate['candidates'][0]['place_id'] #extracts the first poi
            details = self.gmaps.place(place_id)
            latlon = details['result']['geometry']['location']
            return {'place_id': place_id, 'latlon': latlon,'details':details}
        except:
            print('Error: Possibly, didnt find enough candidates?')
        
    def generate_itinerary_and_directions(self, pois):
        
        mean_loc = self.get_mean_location(pois)
        itin = self.generate_itinerary(pois)
        
        result = dict()
        result['mean_loc'] = mean_loc
        result['ordered_pois'] = [pois[el] for el in itin['path']]
        
        result['transits'] = []
        
        result['transits'] += [self.get_transit_directions(mean_loc, pois[itin['path'][0]])]
        
        for i in range(len(pois)-1):
            result['transits'] += [self.get_transit_directions(pois[itin['path'][i]],pois[itin['path'][i+1]])]
            
        result['transits'] += [self.get_transit_directions(pois[itin['path'][-1]], mean_loc)]
                
        return result
        
        
    def get_transit_directions(self,origin, destination, location_bias=None, mode='transit', summarise = True):
        #assume origin is either in a amadeus style or is a text entry for google maps search
        
        if location_bias is None:
            try:
                location_bias = self.poi_latlon
                
            except:
                pass
        
        if type(origin) is str:
            origin_details = self.get_details(origin, location_bias=location_bias)
            origin_latlon = origin_details['latlon']
        elif 'latitude' in origin.keys():
            origin_latlon = origin
        else:
            #assumes it is a amadeus location
            origin_latlon = origin['geoCode']
        
        if type(destination) is str:
            dest_details = self.get_details(destination, location_bias=location_bias)
            dest_latlon = dest_details['latlon']
        elif 'latitude' in destination.keys():
            dest_latlon = destination
        else:
            dest_latlon = destination['geoCode']
        
        transit = self.gmaps.directions(origin_latlon,dest_latlon,mode='transit')
        
        if summarise:
            return self.get_transit_summary(transit)
        else:
            return transit
        
        
        
    
        
        
    
    def get_transit_summary(self,transit):
        summary = dict()
        
        summary['total_time'] = transit[0]['legs'][0]['duration']['text']
        
        
        legs = dict()
        for i in range(len(transit[0]['legs'][0]['steps'])):

            dur_string  = transit[0]['legs'][0]['steps'][i]['duration']['text']
            description = transit[0]['legs'][0]['steps'][i]['html_instructions']

            legs[i] = {'duration': dur_string, 'text': description}

        summary['legs'] = legs

        
        return summary
        

In [437]:
for i in range(1):
    print(2)

2


In [359]:
legs=dict()

In [325]:
legs[0] = {'duration': '4 mins', 'text': 'walk'}

In [326]:
legs

{0: {'duration': '4 mins', 'text': 'walk'}}

In [184]:
import os
os.chdir('../data-import/')

from AmadeusClient import AmadeusClient

am = AmadeusClient()

os.chdir('../planner/')

True

In [185]:
am

In [400]:
pois = random.sample(pois, k = 9)
pois

[{'type': 'location',
  'subType': 'POINT_OF_INTEREST',
  'geoCode': {'latitude': 50.092182, 'longitude': 14.449174},
  'name': 'Charleston',
  'category': 'RESTAURANT',
  'tags': ['restaurant'],
  'city': ''},
 {'type': 'location',
  'subType': 'POINT_OF_INTEREST',
  'geoCode': {'latitude': 50.085323, 'longitude': 14.42018},
  'name': 'AnonymouS Bar',
  'category': 'RESTAURANT',
  'tags': ['restaurant', 'bar', 'club', 'nightlife'],
  'city': ''},
 {'type': 'location',
  'subType': 'POINT_OF_INTEREST',
  'geoCode': {'latitude': 50.082363, 'longitude': 14.423106},
  'name': 'Franciscan garden Františkánská zahrada',
  'category': 'SIGHTS',
  'tags': ['garden'],
  'city': ''},
 {'type': 'location',
  'subType': 'POINT_OF_INTEREST',
  'geoCode': {'latitude': 50.08469, 'longitude': 14.423233},
  'name': 'U Provaznice',
  'category': 'RESTAURANT',
  'tags': ['restaurant'],
  'city': ''},
 {'type': 'location',
  'subType': 'POINT_OF_INTEREST',
  'geoCode': {'latitude': 50.089954, 'longitude'

In [453]:
client = GoogleMapsClient()

In [402]:
placeids = client.get_place_ids_from_POIs(pois)

In [456]:
itin = client.generate_itinerary(pois)

In [458]:
itin.keys()

dict_keys(['path', 'dist_matrix', 'travel_time'])

In [461]:
type(itin['path'][0])

numpy.int64

In [409]:
pois[0, 2, 1, 3, 5, 8, 6, 4, 7]

TypeError: 'list' object is not callable

In [422]:
client.get_mean_location(pois)

{'latitude': 50.08881577777778, 'longitude': 14.424650333333334}

In [433]:
len(client.get_transit_directions(client.get_mean_location(pois),pois[path[0]], summarise=False)[0]['legs'])

1

In [454]:
client.generate_itinerary_and_directions(pois)

{'mean_loc': {'latitude': 50.08881577777778, 'longitude': 14.424650333333334},
 'ordered_pois': [{'type': 'location',
   'subType': 'POINT_OF_INTEREST',
   'geoCode': {'latitude': 50.089558, 'longitude': 14.421881},
   'name': "Tretter's",
   'category': 'RESTAURANT',
   'tags': ['restaurant', 'bar', 'nightlife'],
   'city': ''},
  {'type': 'location',
   'subType': 'POINT_OF_INTEREST',
   'geoCode': {'latitude': 50.089893, 'longitude': 14.416262},
   'name': 'Museum of Decorative Arts in Prague',
   'category': 'SIGHTS',
   'tags': ['sightseeing', 'museum', 'sights'],
   'city': ''},
  {'type': 'location',
   'subType': 'POINT_OF_INTEREST',
   'geoCode': {'latitude': 50.089954, 'longitude': 14.415439},
   'name': 'Rudolfinum',
   'category': 'SIGHTS',
   'tags': ['sightseeing', 'restaurant', 'artgallerie', 'sights', 'landmark'],
   'city': ''},
  {'type': 'location',
   'subType': 'POINT_OF_INTEREST',
   'geoCode': {'latitude': 50.085323, 'longitude': 14.42018},
   'name': 'AnonymouS 

In [ ]:
self.get_transit_summary

In [368]:
gmaps.find_place('BER',input_type='textquery')

{'candidates': [{'place_id': 'ChIJq_-Z1yoFdkgRpXtYsZTGvGk'}], 'status': 'OK'}

In [392]:
gmaps.place('ChIJq_-Z1yoFdkgRpXtYsZTGvGk')['result']

{'address_components': [{'long_name': '20',
   'short_name': '20',
   'types': ['street_number']},
  {'long_name': 'Cannon Street',
   'short_name': 'Cannon St',
   'types': ['route']},
  {'long_name': 'London', 'short_name': 'London', 'types': ['postal_town']},
  {'long_name': 'Greater London',
   'short_name': 'Greater London',
   'types': ['administrative_area_level_2', 'political']},
  {'long_name': 'England',
   'short_name': 'England',
   'types': ['administrative_area_level_1', 'political']},
  {'long_name': 'United Kingdom',
   'short_name': 'GB',
   'types': ['country', 'political']},
  {'long_name': 'EC4M 6XD',
   'short_name': 'EC4M 6XD',
   'types': ['postal_code']}],
 'adr_address': '<span class="street-address">20 Cannon St</span>, <span class="locality">London</span> <span class="postal-code">EC4M 6XD</span>, <span class="country-name">UK</span>',
 'formatted_address': '20 Cannon St, London EC4M 6XD, UK',
 'formatted_phone_number': '020 7529 1111',
 'geometry': {'locatio

In [372]:
client.get_details(['ChIJq_-Z1yoFdkgRpXtYsZTGvGk'])

Possibly, didnt find enough candidates?


In [329]:
hotel=client.get_mean_location(pois)

In [330]:
len(pois[:-1])

9

In [260]:
itinerary = client.generate_itinerary(pois = pois[:-1], hotel_lat_lon= hotel)

In [342]:
itinerary

{'path': [4, 0, 2, 1, 7, 3, 5, 6, 8],
 'dist_matrix': {'destination_addresses': ['Týnská 21, 110 00 Staré Město, Czechia',
   'Křižíkova 208/55, 186 00 Karlín, Czechia',
   'Provaznická 388/2, 110 00 Praha-Staré Město, Czechia',
   'Ovocný trh 19, 110 00 Praha 1-Staré Město, Czechia',
   'Valentinská 56/11, 110 00 Praha-Staré Město, Czechia',
   'Letohradská 648/34, 170 00 Praha 7-Holešovice, Czechia',
   'Alšovo nábř. 12, 110 00 Praha 1-Staré Město, Czechia',
   '17. listopadu 2, 110 00 Praha 1-Staré Město, Czechia',
   'Michalská 433/14, 110 00 Praha-Staré Město, Czechia',
   'V Kolkovně 921/3, 110 00 Praha-Staré Město, Czechia'],
  'origin_addresses': ['Týnská 21, 110 00 Staré Město, Czechia',
   'Křižíkova 208/55, 186 00 Karlín, Czechia',
   'Provaznická 388/2, 110 00 Praha-Staré Město, Czechia',
   'Ovocný trh 19, 110 00 Praha 1-Staré Město, Czechia',
   'Valentinská 56/11, 110 00 Praha-Staré Město, Czechia',
   'Letohradská 648/34, 170 00 Praha 7-Holešovice, Czechia',
   'Alšovo 

In [331]:
path = itinerary['path']

In [337]:
path

[4, 0, 2, 1, 7, 3, 5, 6, 8]

In [338]:
hotel

{'latitude': 50.0887656, 'longitude': 14.423813200000001}

In [339]:
pois[path[0]].keys()

dict_keys(['type', 'subType', 'geoCode', 'name', 'category', 'tags', 'city'])

In [340]:
transit_dir = client.get_transit_directions(hotel, pois[path[0]])

In [341]:
transit_dir

{'total_time': '13 mins',
 'legs': {0: {'duration': '5 mins', 'text': 'Walk to Dlouhá třída'},
  1: {'duration': '5 mins', 'text': 'Tram towards Divoká Šárka'},
  2: {'duration': '4 mins',
   'text': 'Walk to Letohradská 648/34, 170 00 Praha 7-Holešovice, Czechia'}}}

In [321]:
for i in range(len(transit_dir[0]['legs'][0]['steps'])):
    dur_string = transit_dir[0]['legs'][0]['steps'][i]['duration']['text']
    description = transit_dir[0]['legs'][0]['steps'][i]['html_instructions']
    
    print(dur_string,': ', description)

print('Total time: ', transit_dir[0]['legs'][0]['duration']['text'])

5 mins :  Walk to Dlouhá třída
5 mins :  Tram towards Divoká Šárka
4 mins :  Walk to Letohradská 648/34, 170 00 Praha 7-Holešovice, Czechia
Total time:  14 mins


In [317]:
transit_dir

[{'bounds': {'northeast': {'lat': 50.09986319999999, 'lng': 14.4332601},
   'southwest': {'lat': 50.08877649999999, 'lng': 14.4238037}},
  'copyrights': 'Map data ©2019 Google',
  'fare': {'currency': 'CZK', 'text': 'CZK\xa024.00', 'value': 24},
  'legs': [{'arrival_time': {'text': '3:03am',
     'time_zone': 'Europe/Prague',
     'value': 1564362238},
    'departure_time': {'text': '2:49am',
     'time_zone': 'Europe/Prague',
     'value': 1564361374},
    'distance': {'text': '2.3 km', 'value': 2336},
    'duration': {'text': '14 mins', 'value': 864},
    'end_address': 'Letohradská 648/34, 170 00 Praha 7-Holešovice, Czechia',
    'end_location': {'lat': 50.0983823, 'lng': 14.4270437},
    'start_address': 'Týnská 21, 110 00 Staré Město, Czechia',
    'start_location': {'lat': 50.08877649999999, 'lng': 14.4238037},
    'steps': [{'distance': {'text': '0.4 km', 'value': 440},
      'duration': {'text': '5 mins', 'value': 325},
      'end_location': {'lat': 50.0905274, 'lng': 14.427838

'14 mins'

In [117]:
gmaps.directions(placeids[0], placeids[1],mode='transit',departure_time=now)

HTTPError: HTTP Error: 400

In [128]:
gmaps.distance_matrix(placeids[0], placeids[1])

HTTPError: HTTP Error: 400

In [67]:
f"point:{pois[0]['geoCode']['latitude']},{pois[0]['geoCode']['longitude']}"

'point:50.085133,14.424776'

In [249]:
candidates = gmaps.find_place(input = pois[0]['name'], input_type='textquery',fields=['place_id'],location_bias=f"point:{pois[0]['geoCode']['latitude']},{pois[0]['geoCode']['longitude']}")
placeid = candidates['candidates'][0]['place_id']

In [250]:
placeid

'ChIJlxSKQqOUC0cROf8ruWvkSOg'

In [70]:
placeid

'ChIJqSCReOuUC0cREEclXxCXQVM'

In [ ]:
gmaps.geocode('')

In [ ]:
deets = gmaps.place(geocode_result[0]['place_id'])

In [37]:
deets = gmaps.place(geocode_result[0]['place_id'])

In [256]:
deets['result']['geometry']['location']

{'lat': 51.52725940000001, 'lng': -0.08702050000000001}

In [41]:
deets['result']['photos']

[{'height': 426,
  'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102939174042935545234/photos">WeWork 41 Corsham St</a>'],
  'photo_reference': 'CmRaAAAAtalm9OZgfCq58PEbGlMHji3yfWr6JIj1Q4XwFELDksszDqADz6k_YVvGL65oCwHXqxSf8ncdjhD_h57BvCzWvSS2a6gSyQ_UEabuDAqpAZRE50YPhEeAALRrljrQ6QZ3EhAEkKot3M3hrQO6uJzKM3W2GhTvlKDwPK2gPGz-QFWb5_XUou32Rw',
  'width': 758},
 {'height': 2640,
  'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102939174042935545234/photos">WeWork 41 Corsham St</a>'],
  'photo_reference': 'CmRaAAAA74t1xX68coAzdO8h-nQuQHQuI5mo9g6FVJpITZMwAxALRRyOZXTYNEeAL5K3oTsV7q77V1Ia28GsqAWeBe4E-tzWPN-gH5DLNE33eIhrMZWQsbjWLYR6sjtZ8lWi0qvbEhD2b3z6SBFQSuqJyK2OJGalGhRuKKc9vseUCnsJ_00HJ6oKpJYYFA',
  'width': 3960},
 {'height': 3024,
  'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109014961715875174436/photos">Amar Patel</a>'],
  'photo_reference': 'CmRaAAAAcWi2XFaF05stQ94oL8ssZY-HdcrZYphbXx7__N0VQ3s-om66G0_ML9PdW3zAn_JiAq8C7s8SU6O

In [43]:
deets['result']['photos'][0]['html_attributions']

['<a href="https://maps.google.com/maps/contrib/102939174042935545234/photos">WeWork 41 Corsham St</a>']

In [44]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://maps.google.com/maps/contrib/102939174042935545234/photos")

In [122]:
pois[0]['geoCode']

{'latitude': 50.085133, 'longitude': 14.424776}

In [124]:
gmaps.directions(pois[0]['geoCode'],pois[1]['geoCode'],mode='transit')

[{'bounds': {'northeast': {'lat': 50.09358, 'lng': 14.4553678},
   'southwest': {'lat': 50.08361, 'lng': 14.42388}},
  'copyrights': 'Map data ©2019 Google',
  'fare': {'currency': 'CZK', 'text': 'CZK\xa024.00', 'value': 24},
  'legs': [{'arrival_time': {'text': '8:19pm',
     'time_zone': 'Europe/Prague',
     'value': 1564337997},
    'departure_time': {'text': '8:06pm',
     'time_zone': 'Europe/Prague',
     'value': 1564337199},
    'distance': {'text': '3.0 km', 'value': 3040},
    'duration': {'text': '13 mins', 'value': 798},
    'end_address': 'Křižíkova 386/105, 186 00 Karlín, Czechia',
    'end_location': {'lat': 50.0935618, 'lng': 14.4553186},
    'start_address': 'Swarovski Boutique, Na Příkopě 15, 110 00 Praha-Staré Město, Czechia',
    'start_location': {'lat': 50.0850945, 'lng': 14.4248348},
    'steps': [{'distance': {'text': '0.2 km', 'value': 183},
      'duration': {'text': '3 mins', 'value': 161},
      'end_location': {'lat': 50.08361, 'lng': 14.42398},
      'htm

In [129]:
origins = [pois[i]['geoCode'] for i in range(len(pois))]
destinations = [pois[i]['geoCode'] for i in range(len(pois))]

gmaps.directions(origins[0],origins[1],mode='transit')

[{'bounds': {'northeast': {'lat': 50.09358, 'lng': 14.4553678},
   'southwest': {'lat': 50.08361, 'lng': 14.42388}},
  'copyrights': 'Map data ©2019 Google',
  'fare': {'currency': 'CZK', 'text': 'CZK\xa024.00', 'value': 24},
  'legs': [{'arrival_time': {'text': '8:34pm',
     'time_zone': 'Europe/Prague',
     'value': 1564338897},
    'departure_time': {'text': '8:21pm',
     'time_zone': 'Europe/Prague',
     'value': 1564338099},
    'distance': {'text': '3.0 km', 'value': 3040},
    'duration': {'text': '13 mins', 'value': 798},
    'end_address': 'Křižíkova 386/105, 186 00 Karlín, Czechia',
    'end_location': {'lat': 50.0935618, 'lng': 14.4553186},
    'start_address': 'Swarovski Boutique, Na Příkopě 15, 110 00 Praha-Staré Město, Czechia',
    'start_location': {'lat': 50.0850945, 'lng': 14.4248348},
    'steps': [{'distance': {'text': '0.2 km', 'value': 183},
      'duration': {'text': '3 mins', 'value': 161},
      'end_location': {'lat': 50.08361, 'lng': 14.42398},
      'htm

In [134]:
dist_matrix = gmaps.distance_matrix(origins,destinations,mode='transit')

In [178]:
pd.DataFrame.from_dict(origins).mean()

latitude     50.087013
longitude    14.430674
dtype: float64

In [179]:
origins

[{'latitude': 50.085133, 'longitude': 14.424776},
 {'latitude': 50.09354, 'longitude': 14.455214},
 {'latitude': 50.08562, 'longitude': 14.431791},
 {'latitude': 50.08641, 'longitude': 14.420797},
 {'latitude': 50.09586, 'longitude': 14.427088},
 {'latitude': 50.08884, 'longitude': 14.419975},
 {'latitude': 50.086445, 'longitude': 14.42011},
 {'latitude': 50.083973, 'longitude': 14.414278},
 {'latitude': 50.081203, 'longitude': 14.444147},
 {'latitude': 50.08311, 'longitude': 14.44856}]

In [148]:
ll = [[dist_matrix['rows'][o]['elements'][d]['duration']['value'] for d in range(len(destinations))] for o in range(len(origins))]

In [149]:
ll

[[0, 798, 478, 300, 1094, 416, 326, 683, 1152, 1138],
 [867, 0, 840, 1019, 1349, 1244, 1086, 1276, 1832, 1511],
 [472, 776, 0, 704, 958, 839, 771, 736, 902, 781],
 [313, 970, 672, 0, 955, 210, 65, 538, 1424, 1303],
 [1001, 1132, 813, 995, 0, 773, 976, 773, 1438, 1317],
 [444, 1404, 869, 224, 914, 0, 224, 567, 1378, 1554],
 [346, 1043, 745, 70, 961, 215, 0, 499, 1391, 1438],
 [714, 1358, 767, 565, 1140, 658, 512, 0, 1519, 1398],
 [972, 1377, 743, 1129, 1552, 1175, 1196, 1329, 0, 361],
 [891, 1381, 575, 1129, 1315, 1279, 1285, 1161, 367, 0]]

In [141]:
dist_matrix['rows'][0]['elements'][1]

{'distance': {'text': '3.0 km', 'value': 3040},
 'duration': {'text': '13 mins', 'value': 798},
 'fare': {'currency': 'CZK', 'text': 'CZK\xa024.00', 'value': 24},
 'status': 'OK'}

In [154]:
#algo to find best path through some weights
#assume there is a visited list, where you dont want to go to 
#assume you start at the origin, the next node is simply the next nearest node. 

path = [0]
visited = set([0])
current_location = 0

In [153]:
visited

{0}

In [155]:
ll[current_location]

[0, 798, 478, 300, 1094, 416, 326, 683, 1152, 1138]

In [157]:
min(ll[current_location])

0

In [163]:
import numpy as np
lla = np.array(ll)

In [164]:
lla

array([[   0,  798,  478,  300, 1094,  416,  326,  683, 1152, 1138],
       [ 867,    0,  840, 1019, 1349, 1244, 1086, 1276, 1832, 1511],
       [ 472,  776,    0,  704,  958,  839,  771,  736,  902,  781],
       [ 313,  970,  672,    0,  955,  210,   65,  538, 1424, 1303],
       [1001, 1132,  813,  995,    0,  773,  976,  773, 1438, 1317],
       [ 444, 1404,  869,  224,  914,    0,  224,  567, 1378, 1554],
       [ 346, 1043,  745,   70,  961,  215,    0,  499, 1391, 1438],
       [ 714, 1358,  767,  565, 1140,  658,  512,    0, 1519, 1398],
       [ 972, 1377,  743, 1129, 1552, 1175, 1196, 1329,    0,  361],
       [ 891, 1381,  575, 1129, 1315, 1279, 1285, 1161,  367,    0]])

In [165]:
np.factorial(10)

AttributeError: module 'numpy' has no attribute 'factorial'

In [174]:
import tsp

In [167]:
r = range(len(ll))

In [168]:
r

range(0, 10)

In [169]:
ll

[[0, 798, 478, 300, 1094, 416, 326, 683, 1152, 1138],
 [867, 0, 840, 1019, 1349, 1244, 1086, 1276, 1832, 1511],
 [472, 776, 0, 704, 958, 839, 771, 736, 902, 781],
 [313, 970, 672, 0, 955, 210, 65, 538, 1424, 1303],
 [1001, 1132, 813, 995, 0, 773, 976, 773, 1438, 1317],
 [444, 1404, 869, 224, 914, 0, 224, 567, 1378, 1554],
 [346, 1043, 745, 70, 961, 215, 0, 499, 1391, 1438],
 [714, 1358, 767, 565, 1140, 658, 512, 0, 1519, 1398],
 [972, 1377, 743, 1129, 1552, 1175, 1196, 1329, 0, 361],
 [891, 1381, 575, 1129, 1315, 1279, 1285, 1161, 367, 0]]

In [170]:
ll[2][3]

704

In [171]:
dist = {(i,j): ll[i][j] for i in r for j in r}

In [175]:
print(tsp.tsp(r, dist))

(6222.0, [0, 8, 9, 2, 1, 4, 7, 6, 3, 5])


In [180]:
path = [0, 8, 9, 2, 1, 4, 7, 6, 3, 5]

In [182]:
[loc-1 for loc in path[1:]]

[7, 8, 1, 0, 3, 6, 5, 2, 4]